In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import Sequential, layers
from tensorflow.keras.preprocessing.image import load_img 
import os
import numpy as np
# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

flowers_dir = "./flowers_1"
flower_labels = sorted(os.listdir(flowers_dir))
flower_labels


img_height = 299
img_width = 299
# img_height = 224
# img_width = 224


model = tf.keras.Sequential(
    [
        layers.experimental.preprocessing.Rescaling(
            1.0 / 255, input_shape=(img_height, img_width, 3)
        ),
        hub.KerasLayer(
            "https://tfhub.dev/google/imagenet/inception_v3/feature_vector/4",
            # "https://www.kaggle.com/models/tensorflow/resnet-50/frameworks/TensorFlow2/variations/classification/versions/1",
            trainable=False,
        ),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(5, activation="softmax"),
        # tf.keras.layers.Dense(512, activation="relu"),
        # tf.keras.layers.Dense(100, activation="softmax"),
    ]
)


model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics="accuracy"
)

model.load_weights("./flowers_test.h5")

In [9]:
file_url = input("image url: ")

file_path = None

try:
    file_path = tf.keras.utils.get_file(
        origin=file_url, cache_dir="./downloaded-images"
    )
except: 
   None

if file_path != None: 
  img = tf.keras.utils.load_img(file_path, target_size=(img_height, img_width))
  img_array = tf.keras.utils.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)

  predictions = model.predict(img_array)
  score = tf.nn.softmax(predictions[0])

  max_score = 100 * np.max(score)

  if max_score < 35:
     print("----- unsure, best guess {} {:.2f} -----".format(
          flower_labels[np.argmax(score)], 100 * np.max(score)
      ))
  else:
    print(
        "----- {} {:.2f} -----".format(
            flower_labels[np.argmax(score)], 100 * np.max(score)
        )
    )


1/1 [==============================] - 0s 15ms/step
----- sunflower 40.35 -----
